In [ ]:
! pip install GPy

In [ ]:
! pip install GPyOpt

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
# %tensorflow_version 1.x
import tensorflow as tf
import keras
import keras.backend as K
from keras import layers
from keras.models import Sequential,Model

from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.layers import Dense
import gc
from keras.layers.merge import concatenate
import csv
import math

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight')


import numpy as np
from numpy.random import rand
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from random import randrange
import time


import GPy, GPyOpt

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def single_point_smape(actual, forecast):
    return (100/len(actual)) * (np.sum(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast))))
def smape(actual, forecast):
    sum_smape=0
    for i in range(len(actual)):
        sum_smape+=single_point_smape(actual[i],forecast[i])
    return sum_smape/len(actual)

In [ ]:
def create_dataset(dataset, look_back,look_forward=7):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-look_forward+1):
    
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        a=dataset[(i + look_back):(i + look_back+look_forward), 0]
        dataY.append(a)
    return np.array(dataX), np.array(dataY)

In [ ]:
def create_dataset_features(dataset, look_back,look_forward=7):
    sample = []
    sample = [float(item) for item in dataset[look_back-5] if math.isnan(item)==False]
    print(len(sample))
    sample=np.array(sample)
    sample = sample.reshape(sample.shape[0],1)
    scaler = MinMaxScaler(copy=True,feature_range=(0, 1))
    sample = scaler.fit_transform(sample)
    
    sample=sample.reshape(11,int(len(sample)/11))
    sample=sample.T
    sample=np.array(sample)
    sample=sample[0:-7]
    
    return sample

In [ ]:
def create_sample(dataX,dataY,one_sample,look_back,look_forward):
    testsize=35
    val_test_size=int(len(dataX)*20/100)
    train_size=(len(one_sample)-look_forward)-testsize-look_back+1
    print (train_size)########## manual setting of train test
    trainX, testX = dataX[0:train_size,:], dataX[train_size+look_forward-1:,:]
    trainY, testY = dataY[0:train_size,:], dataY[train_size+look_forward-1:,:]

    valsize=33

    valX, valY=trainX[train_size-valsize+look_forward-1:train_size,:],trainY[train_size-valsize+look_forward-1:train_size]

    trainX = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
    valX= np.reshape(valX, (valX.shape[0],1, valX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0],1, testX.shape[1]))
    return np.array(trainX),np.array(valX),np.array(testX),np.array(trainY),np.array(valY),np.array(testY)

In [ ]:
all_train = pd.read_csv('/kaggle/input/indiacovid-reza-ali/Top_20_series.csv')
all_train

In [ ]:
all_train=np.array(all_train)
all_train

In [ ]:
features = pd.read_csv('/kaggle/input/us-covid19-with-feature/US_features5.csv')
features

In [ ]:
features=features.T
features

In [ ]:
features=np.array(features)
features

In [ ]:
one_sample=[float(item) for item in all_train[0,3:] if math.isnan(item)==False]
print(len(one_sample))

In [ ]:
look_back=0

In [ ]:
def all_pre_process(one_sample,features,lag,look_forward,):
    look_forward=7
    look_back=lag
    
    
    one_sample=np.array(one_sample)
    one_sample = one_sample.astype('int64')
    one_sample = one_sample.reshape(one_sample.shape[0],1)
    

    dataX_s, dataY_s = create_dataset(one_sample,look_back,look_forward)
    trainX,valX,testX,trainY,valY,testY = create_sample(dataX_s,dataY_s,one_sample,look_back,look_forward)
    
    dataX_features = create_dataset_features(features,look_back,look_forward)
    
   
    
    
    trainX_f = dataX_features[0:len(trainX)]
    valX_f   = dataX_features[len(trainX)-len(valX):len(trainX)]
    testX_f  = dataX_features[-len(testX):]
    
    trainX_f = np.reshape(trainX_f, (trainX_f.shape[0],1, trainX_f.shape[1]))
    valX_f= np.reshape(valX_f, (valX_f.shape[0],1, valX_f.shape[1]))
    testX_f = np.reshape(testX_f, (testX_f.shape[0],1, testX_f.shape[1]))



    return trainX,valX,testX,trainY,valY,testY,trainX_f,valX_f,testX_f


In [ ]:
def all_hyperparametrs(hyper_parametrs):
    if hyper_parametrs[0]==0:
        conv_activation=None
    elif hyper_parametrs[0]==1:
        conv_activation="relu"
    elif hyper_parametrs[0]==2:
        conv_activation="linear"
    elif hyper_parametrs[0]==3:
        conv_activation="tanh"
    
    kernel_size = int(hyper_parametrs[1])
    stride = int(hyper_parametrs[2])
    conv_neurons = int(hyper_parametrs[3])
    learning_rate = hyper_parametrs[4]
    
    
    if hyper_parametrs[5]==0:
        denselayer_activation=None
    elif hyper_parametrs[5]==1:
        denselayer_activation="relu"
    elif hyper_parametrs[5]==2:
        denselayer_activation="linear"
    
    
    if hyper_parametrs[6]==0:
        feature_conv_activation=None
    elif hyper_parametrs[6]==1:
        feature_conv_activation="relu"
    elif hyper_parametrs[6]==2:
        feature_conv_activation="linear"
    elif hyper_parametrs[6]==3:
        feature_conv_activation="tanh"
    
    
    feature_kernel_size = int(hyper_parametrs[7])
    feature_stride = int(hyper_parametrs[8])
    feature_conv_neurons = int(hyper_parametrs[9])
    
    if hyper_parametrs[10]==0:
        output_activation=None
    elif hyper_parametrs[10]==1:
        output_activation="relu"
    elif hyper_parametrs[10]==2:
        output_activation="linear"
    
    return conv_activation,kernel_size,stride,conv_neurons,learning_rate,denselayer_activation,feature_conv_activation,feature_kernel_size,feature_stride,feature_conv_neurons,output_activation

In [ ]:
trainX,valX,testX,trainY,valY,testY,trainX_f,valX_f,testX_f=all_pre_process(one_sample,features,15,7)

In [ ]:
trainX.shape

In [ ]:
trainX_f.shape

In [ ]:
def M_Model(shape2=(1,11,)):
    input1 = layers.Input(shape=(1,15,))
    input2 = layers.Input(shape=shape2)

    conv1 = layers.Conv1D(64,
                              strides=2,
                              kernel_size=4,
                              activation=None,
                              padding="same",)(input1)
                          
    conv2 = layers.Conv1D(64,
                              strides=2,
                              kernel_size=4,
                              activation='linear',
                              padding="same",)(input2)                    

                          
    
    flatt1 = keras.layers.Flatten()(conv1)
    flatt2 = keras.layers.Flatten()(conv2) 
    concat=concatenate([flatt1,flatt2])

    fully_connected = Dense(100,activation='linear')(concat)

    hidden3 = Dense(7,activation='linear')(fully_connected)

    model = Model(inputs=[input1,input2],outputs=hidden3)
    opt=tf.optimizers.Adam(lr=0.0001)
    model.compile(loss="mse",optimizer=opt,metrics=["mse"])
    return model

In [ ]:
# error rate
def error_rate(xtrain, ytrain, x, opts):
    # parameters
    fold = opts['fold']
    xtrain = fold['xt']
    ytrain = fold['yt']
    xv = fold['xv']
    yv = fold['yv']
    xt_f = fold['xt_f']
    xv_f = fold['xv_f']

    # Define selected features
    xtrain_f = xt_f[:, :, x == 1]
    xvalid_f = xv_f[:, :, x == 1]
    # Training
    mdl     = M_Model(shape2=(1, xtrain_f.shape[2],))
    history = mdl.fit(
        [xtrain,xtrain_f],
        y = ytrain,
        batch_size = 1,
        epochs = 500,
        validation_data=([xv,xvalid_f], yv),
        verbose=0,
        callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=30, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=50, factor=0.5, min_lr=1e-5)
        ]
        ).history

    # Prediction
    results = mdl.predict([valX,xvalid_f],batch_size=16, verbose=0)
    results = results.astype('int64')
    results = results.round()
    val_RMSE = math.sqrt(mean_squared_error(valY, results[:]))    
    return val_RMSE

In [ ]:
# Error rate & Feature size
def Fun(xtrain, ytrain, x, opts):
    # parameters
    alpha = 0.99
    beta = 1 - alpha
    # original feature size
    max_feat = len(x)
    # Number of selected features
    num_feat = np.sum(x == 1)
    # Solve if no feature selected
    if num_feat == 0:
        cost = 100000000000000
    else:
        # Get error rate
        error = error_rate(xtrain, ytrain, x, opts)
        # Objective function
        cost = alpha * error + beta * (num_feat / max_feat)
    return cost

In [ ]:
def init_position(lb, ub, N, dim):
    X = np.zeros([N, dim], dtype='float')
    for i in range(N):
        for d in range(dim):
            X[i,d] = lb[0,d] + (ub[0,d] - lb[0,d]) * rand()        
    
    return X

In [ ]:
def binary_conversion(X, thres, N, dim):
    Xbin = np.zeros([N, dim], dtype='int')
    for i in range(N):
        for d in range(dim):
            if X[i,d] > thres:
                Xbin[i,d] = 1
            else:
                Xbin[i,d] = 0
    
    return Xbin

In [ ]:
def boundary(x, lb, ub):
    if x < lb:
        x = lb
    if x > ub:
        x = ub
    
    return x

In [ ]:
def roulette_wheel(prob):
    num = len(prob)
    C   = np.cumsum(prob)
    P   = rand()
    for i in range(num):
        if C[i] > P:
            index = i;
            break
    
    return index

In [ ]:
def jfs(xtrain, ytrain, opts):
    # Parameters
    ub = 1
    lb = 0
    thres = 0.5
    CR = 0.8  # crossover rate
    MR = 0.01 # mutation rate
    
    N = opts['N']
    max_iter = opts['T']
    if 'CR' in opts:
        CR = opts['CR']
    if 'MR' in opts:
        MR = opts['MR']
    
    # Dimension
    fold = opts['fold']
    xtrain_f = fold['xt_f']
    dim = np.size(xtrain_f, 2)
    if np.size(lb) == 1:
        ub = ub * np.ones([1, dim], dtype='float')
        lb = lb * np.ones([1, dim], dtype='float')
    
    # Initialize position
    X = init_position(lb, ub, N, dim)
    
    # Binary conversion
    Xbin = binary_conversion(X, thres, N, dim)
    
    # Fitness at first iteration
    fit = np.zeros([N, 1], dtype='float')
    Xgb = np.zeros([1, dim], dtype='float')
    fitG = float('inf')
    
    for i in range(N):
        fit[i,0] = Fun(xtrain, ytrain, Xbin[i, :], opts)
        if fit[i,0] < fitG:
            Xgb[0,:] = X[i,:]
            fitG = fit[i,0]
            
    # Pre
    curve = np.zeros([1, max_iter], dtype='float') 
    t     = 0
    
    curve[0,t] = fitG.copy()
    print("Generation:", t + 1)
    print("Best (JA + GA):", curve[0,t])
    t += 1
    
    while t < max_iter:
        N = len(fit)
        Xnew = np.zeros([N, dim], dtype='float')
        
        # Identify best & worst in population
        idx_max = np.argmax(fit)
        Xw = X[idx_max, np.newaxis,:].copy()
        idx_min = np.argmin(fit)
        Xb = X[idx_min, np.newaxis,:].copy()
        
        for i in range(N):
            for d in range(dim):
                r1 = rand()
                r2 = rand()
                # Position update (1)
                Xnew[i,d] = X[i,d] + r1 * (Xb[0,d] - abs(X[i,d])) - r2 * (Xw[0,d] - abs(X[i,d]))
                # Boundary
                Xnew[i,d] = boundary(Xnew[i,d], lb[0,d], ub[0,d])
        
        # Binary conversion
        Xbin = binary_conversion(Xnew, thres, N, dim)
        
        # Greedy selection
        for i in range(N):
            Fnew = Fun(xtrain, ytrain, Xbin[i,:], opts)
            if Fnew < fit[i,0]:
                X[i,:] = Xnew[i,:]
                fit[i,0] = Fnew
                
            if fit[i,0] < fitG:
                Xgb[0,:] = X[i,:]
                fitG = fit[i,0]
            
        # Probability
        inv_fit = 1 / (1 + fit)
        prob = inv_fit / np.sum(inv_fit)
        # Number of crossovers
        Nc = 0
        for i in range(N):
            if rand() < CR:
                Nc += 1
        
        x1 = np.zeros([Nc, dim], dtype='int')
        x2 = np.zeros([Nc, dim], dtype='int')
        for i in range(Nc):
            # Parent selection
            k1  = roulette_wheel(prob)
            k2  = roulette_wheel(prob)
            P1 = X[k1, :].copy()
            P2 = X[k2, :].copy()
            # Random one dimension from 1 to dim
            index = np.random.randint(low=1, high=dim-1)
            # Crossover
            x1[i, :] = np.concatenate((P1[0:index], P2[index:]))
            x2[i, :] = np.concatenate((P2[0:index], P1[index:]))
            # Mutation
            for d in range(dim):
                if rand() < MR:
                    x1[i,d] = 1 - x1[i,d]
                
                if rand() < MR:
                    x2[i,d] = 1 - x2[i,d]
        
        # Merge two group into one
        XnewGA = np.concatenate((x1, x2), axis=0)
        
        # Fitness
        FnewGA = np.zeros([2 * Nc, 1], dtype='float')
        for i in range(2 * Nc):
            FnewGA[i,0] = Fun(xtrain, ytrain, XnewGA[i, :], opts)
            if FnewGA[i,0] < fitG:
                for j in range(N):
                    if fitG == fit[j, 0]: 
                        Xgb[0,:] = XnewGA[i,:]
                        fitG = FnewGA[i,0]
                        fit[j,0] = FnewGA[i,0]
                        X[j,:] = XnewGA[i,:]
        
            
        # Store result
        curve[0,t] = fitG.copy()
        print("Generation:", t + 1)
        print("Best (JA + GA):", curve[0,t])
        t += 1
    
    # Best feature subset
    Gbin       = binary_conversion(Xgb, thres, 1, dim) 
    Gbin       = Gbin.reshape(dim)
    pos        = np.asarray(range(0, dim))    
    sel_index  = pos[Gbin == 1]
    num_feat   = len(sel_index)
    # Create dictionary
    ja_data = {'sf': sel_index, 'c': curve, 'nf': num_feat}
    
    return ja_data  

In [ ]:
fold = {'xt':trainX, 'yt':trainY, 'xv':valX, 'yv':valY, 'xt_f':trainX_f, 'xv_f':valX_f}

In [ ]:
# parameter
N     = 10    # number of salps
T     = 10   # maximum number of iterations
maxLt = 10    # maximum iteration for local search algorithm
opts  = {'fold':fold, 'N':N, 'T':T, 'maxLt':maxLt}

In [ ]:
np.save('trainX.npy', trainX)
np.save('trainY.npy', trainY)
np.save('valX.npy', valX)
np.save('valY.npy', valY)
np.save('trainX_f.npy', trainX_f)
np.save('valX_f.npy', valX_f)
np.save('testX_f.npy', testX_f)

In [ ]:
# perform feature selection
start_time = time.time()
fmdl  = jfs(trainX, trainY, opts)
print("Run Time --- %s seconds ---" % (time.time() - start_time))

sf    = fmdl['sf']

# model with selected features
x_train_f   = trainX_f[:, :,  sf]
x_test_f   = testX_f[:, :, sf]
valX_f = valX_f[:, :, sf]
    
# Training
mdl     = M_Model(shape2=(1, x_train_f.shape[2],))
history = mdl.fit(
   [trainX,x_train_f],
   y = trainY,
   batch_size = 1,
   epochs = 500,
   validation_data=([valX,valX_f], valY),
   verbose=0,
   callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=30, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=50, factor=0.5, min_lr=1e-5)
        ]
        ).history

# Prediction
results = mdl.predict([testX,x_test_f],batch_size=16, verbose=0)
results = results.astype('int64')
results = results.round()
r_y=results[0: : 7]
t_y=testY[0: : 7]
test_RMSE = math.sqrt(mean_squared_error(t_y, r_y[:]))
test_SMAPE = smape(t_y, r_y[:])

print("RMSE:", test_RMSE)
print("SMAPE:", test_SMAPE)
    
# number of selected features
num_feat = fmdl['nf']
print("Feature Size:", num_feat)

# plot convergence
curve   = fmdl['c']
curve   = curve.reshape(np.size(curve,1))
x       = np.arange(0, opts['T'], 1.0) + 1.0

fig, ax = plt.subplots()
ax.plot(x, curve, 'o-')
ax.set_xlabel('Number of Iterations')
ax.set_ylabel('Fitness')
ax.set_title('JA + GA')
ax.grid()
plt.show()

In [ ]:
sf

In [ ]:
np.save('Feature_Index_15.npy', sf)